In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
! pip install nltk

In [75]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
import numpy as np
from datasets import load_dataset

import nltk
nltk.download("punkt_tab")
nltk.download("punkt")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from datasets import load_dataset

dataset_name = "ibm-research/duorc"
subset_name = "SelfRC"
raw_dataset = load_dataset(dataset_name, subset_name, split="test")
dataset = raw_dataset.shuffle(seed=42).select(range(12559))
test_dataset = raw_dataset.shuffle(seed=42).select(range(2000))
print("Dataset sample:", dataset[0])
print("Total records in subset:", len(dataset))


Dataset sample: {'plot_id': '/m/06r6nf', 'plot': 'Adam Hansen (Matt O\'Leary) and his best friend Duffy (Jake Epstein) have some tickets for the Headless Horseman concert, and his sister Chelsea (Laura Vandervoort) has a date with her dreamy boyfriend Peter. The only problem is, they are both grounded: Adam is punished because he did not do his homework, instead making up a story using an article from the magazine The Weekly Secret, and Chelsea because she called Adam a dweeb, which their mother, Lynette (Caroline Rhea), happened to hear. Chelsea and Adam will do whatever it takes to get their mother out of the house, even if it includes a chance meeting with a very mysterious man named Dimitri. Everything seems to go according to plan, until their little brother Taylor (Myles Jeffrey) realizes that Dimitri is a vampire.\nHis brother and sister do not believe Taylor, so he calls Malachi Van Helsing (Robert Carradine), the vampire hunter. The night that their mother goes out with Dimitr

In [ ]:
def format_qa_example(example):
    plot = example["plot"]
    title = example["title"]
    question = example["question"]
    answers_list = example["answers"]
    answer = answers_list[0] if answers_list else "I'm sorry, but the plot does not provide an answer."

    prompt = f'''
    In the movie "{title}"
    The plot was as follows: {plot}
    Question: {question}
    Answer (in a concise phrase):
    '''.strip()

    return {
        "prompt": prompt,
        "target": answer,
    }

In [ ]:
def count_words(text):
    return len(text.split())

def remove_long_samples(dataset, threshold):
    return dataset.filter(lambda x: count_words(x["prompt"] + " " + x["target"]) <= threshold)

In [ ]:
test_dataset = test_dataset.map(format_qa_example)
test_dataset  = remove_long_samples(test_dataset, threshold=1800)
print(f"After filtering, test size: {len(test_dataset)}")

After filtering, test size: 1987


In [80]:
test_dataset.select(range(300)).to_pandas()

,plot_id,plot,title,question_id,question,answers,no_answer,prompt,target
0,/m/06r6nf,Adam Hansen (Matt O'Leary) and his best friend...,Mom's Got a Date With a Vampire,5d9fff6a-0507-fba1-b41c-21d7ad650f9a,Who made up the spoon test?,[Adam],False,"In the movie ""Mom's Got a Date With a Vampire""...",Adam
1,/m/0g9wdmc,The film begins in September 2008 (opening aga...,The Iron Lady,286ea526-555c-a2a7-807d-9b1ecd91c060,What is the Community Charge also known as?,[Poll Tax],False,"In the movie ""The Iron Lady""\n The plot was...",Poll Tax
2,/m/0crst6n,"Two pairs of friends, Matteo and Francesca &am...",David's Birthday,d1b01647-6d27-f3f8-beef-b566edbe86fa,Where does the beach house locate?,[At the foot of Mount Circeo.],False,"In the movie ""David's Birthday""\n The plot ...",At the foot of Mount Circeo.
3,/m/04jplwp,This article's plot summary may be too long or...,Nine,77854dc8-b9ee-adec-a1b3-d727a9c30319,Luisa is promised the house will be filled wit...,[Flowers.],False,"In the movie ""Nine""\n The plot was as follo...",Flowers.
4,/m/06z5rr,Teenage science genius Paul Conway (Matthew La...,Deadly Friend,02453a75-26d2-5742-c3e4-1df817bb67bb,What is the name of Paul's mother?,[Jeannie],False,"In the movie ""Deadly Friend""\n The plot was...",Jeannie
...,...,...,...,...,...,...,...,...,...
295,/m/02qsqmq,"Annabelle Fritton (Talulah Riley), an uptight ...",St. Trinian's,29de95f4-284d-6109-90fb-e4f8c4fffeae,What sport does Annabelle begin to play?,[Hockey],False,"In the movie ""St. Trinian's""\n The plot was...",Hockey
296,/m/098dmr,The magically musical duo of Goopy Gyne and Ba...,Hirak Rajar Deshe,797d2d3e-d0dd-cae3-c2ab-f9739f3518dd,What was the treasury guarded by?,[A tiger.],False,"In the movie ""Hirak Rajar Deshe""\n The plot...",A tiger.
297,/m/03gy1gr,Sherman 'Tank' Turner (Dane Cook) is a help li...,My Best Friend's Girl,3133fcf4-efc9-7e91-17fe-6a3a9ee3ce0a,Tank shares an apartment with whom?,[Dustin.],False,"In the movie ""My Best Friend's Girl""\n The ...",Dustin.
298,/m/059rc,This article's plot summary may be too long or...,Natural Born Killers,e30424fa-560a-37d4-f594-a2e6d7988aca,Who is captured and beaten by police?,[Mallory],False,"In the movie ""Natural Born Killers""\n The p...",Mallory


In [ ]:
from collections import defaultdict
from nltk.tokenize import sent_tokenize

movie_sentences_map = defaultdict(list)

for item in dataset:
    title = item["title"]
    plot = item["plot"]
    movie_sentences_map[title].append(plot)

for title in movie_sentences_map:
    combined_plot = " ".join(movie_sentences_map[title])
    sentences = sent_tokenize(combined_plot)
    movie_sentences_map[title] = sentences

some_title = list(movie_sentences_map.keys())[0]
print("Title:", some_title)
print("Number of sentences in plot:", len(movie_sentences_map[some_title]))
print("Sample sentences:", movie_sentences_map[some_title][:3])


Title: Mom's Got a Date With a Vampire
Number of sentences in plot: 162
Sample sentences: ["Adam Hansen (Matt O'Leary) and his best friend Duffy (Jake Epstein) have some tickets for the Headless Horseman concert, and his sister Chelsea (Laura Vandervoort) has a date with her dreamy boyfriend Peter.", 'The only problem is, they are both grounded: Adam is punished because he did not do his homework, instead making up a story using an article from the magazine The Weekly Secret, and Chelsea because she called Adam a dweeb, which their mother, Lynette (Caroline Rhea), happened to hear.', 'Chelsea and Adam will do whatever it takes to get their mother out of the house, even if it includes a chance meeting with a very mysterious man named Dimitri.']


 ### Build a Dictionary of Movie Titles → Plot Sentences

In [ ]:
from collections import defaultdict
from nltk.tokenize import sent_tokenize

movie_sentences_map = defaultdict(list)

for item in dataset:
    title = item["title"]
    plot = item["plot"]
    movie_sentences_map[title].append(plot)

for title in movie_sentences_map:
    combined_plot = " ".join(movie_sentences_map[title])
    sentences = sent_tokenize(combined_plot)
    movie_sentences_map[title] = sentences

some_title = list(movie_sentences_map.keys())[0]
print("Title:", some_title)
print("Number of sentences in plot:", len(movie_sentences_map[some_title]))
print("Sample sentences:", movie_sentences_map[some_title][:3])


Title: Mom's Got a Date With a Vampire
Number of sentences in plot: 162
Sample sentences: ["Adam Hansen (Matt O'Leary) and his best friend Duffy (Jake Epstein) have some tickets for the Headless Horseman concert, and his sister Chelsea (Laura Vandervoort) has a date with her dreamy boyfriend Peter.", 'The only problem is, they are both grounded: Adam is punished because he did not do his homework, instead making up a story using an article from the magazine The Weekly Secret, and Chelsea because she called Adam a dweeb, which their mother, Lynette (Caroline Rhea), happened to hear.', 'Chelsea and Adam will do whatever it takes to get their mother out of the house, even if it includes a chance meeting with a very mysterious man named Dimitri.']


### Embed Movie Titles & Plot Sentences (for Retrieval)

In [ ]:
#!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('all-MiniLM-L6-v2')
movie_titles = list(movie_sentences_map.keys())
title_embeddings = model.encode(movie_titles, convert_to_tensor=True, show_progress_bar=True)
movie_database = {}

for title in movie_titles:
    sentences = movie_sentences_map[title]
    sentence_embs = model.encode(sentences, convert_to_tensor=True)
    movie_database[title] = {
        "sentences": sentences,
        "embeddings": sentence_embs
    }

print(f"Embedded {len(movie_titles)} movie titles.")

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Embedded 1010 movie titles.


In [ ]:
def match_movie_title(query_title):
    query_emb = model.encode(query_title, convert_to_tensor=True)
    scores = util.cos_sim(query_emb, title_embeddings)[0]
    best_idx = torch.argmax(scores).item()
    best_title = movie_titles[best_idx]
    return best_title


In [ ]:
def retrieve_sentences_from_movie(movie_title, question, top_k=3):
    question_emb = model.encode(question, convert_to_tensor=True)
    sentence_embs = movie_database[movie_title]["embeddings"]
    sentences = movie_database[movie_title]["sentences"]
    scores = util.cos_sim(question_emb, sentence_embs)[0]
    top_results = torch.topk(scores, k=min(top_k, len(sentences)))
    top_indices = top_results.indices.cpu().numpy()
    best_sentences = [sentences[idx] for idx in top_indices]
    return best_sentences


In [ ]:
augmented_data = []

TOP_K = 5

for item in test_dataset:
    user_title = item["title"]              
    question = item["question"]
    answers = item["answers"] or ["No answer found."]  
    matched_title = match_movie_title(user_title)
    best_sentences = retrieve_sentences_from_movie(matched_title, question, top_k=TOP_K)
    context_str = "\n".join(best_sentences)
    prompt_text = f"""In the movie "{matched_title}"
Question: {question}
Context:
{context_str}

Answer (in a concise phrase):
"""
    target_answer = " / ".join(answers)

    augmented_data.append({
        "movie_title": user_title,
        "matched_title": matched_title,
        "original_question": question,
        "prompt": prompt_text,
        "target_answer": target_answer
    })

len(augmented_data)


1987

In [ ]:
from datasets import Dataset
test_data = Dataset.from_list(augmented_data)

In [53]:
test_data.select(range(5)).to_pandas()

,movie_title,matched_title,original_question,prompt,target_answer
0,Mom's Got a Date With a Vampire,Mom's Got a Date With a Vampire,Who made up the spoon test?,"In the movie ""Mom's Got a Date With a Vampire""...",Adam
1,The Iron Lady,The Iron Lady,What is the Community Charge also known as?,"In the movie ""The Iron Lady""\nQuestion: What i...",Poll Tax
2,David's Birthday,David's Birthday,Where does the beach house locate?,"In the movie ""David's Birthday""\nQuestion: Whe...",At the foot of Mount Circeo.
3,Nine,Nine,Luisa is promised the house will be filled wit...,"In the movie ""Nine""\nQuestion: Luisa is promis...",Flowers.
4,Deadly Friend,Deadly Friend,What is the name of Paul's mother?,"In the movie ""Deadly Friend""\nQuestion: What i...",Jeannie


In [ ]:
save_dir = "/content/drive/MyDrive/projet_IA/test_data"
test_data.save_to_disk(save_dir)

Saving the dataset (0/1 shards):   0%|          | 0/1987 [00:00<?, ? examples/s]

## inference test

In [1]:
!pip install unsloth
!pip install evaluate
!pip install rouge_score
!pip install -U bitsandbytes

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets, load_from_disk
from transformers import (
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from unsloth import FastLanguageModel 
import numpy as np
import pandas as pd
from google.colab import drive
import time  

In [59]:
save_dir = "/content/drive/MyDrive/projet_IA/test_data"
test_data = load_from_disk(save_dir)

In [ ]:
drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/projet_IA/finetuned-qwen2-3B-DPO-merged-2403"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
max_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_length,
    dtype=None,
    load_in_4bit=True,
)
tokenizer.padding_side = "left"
model = FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.18: Fast Qwen2 patching. Transformers: 4.50.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_example(example, tokenizer, max_length=2048):
    prompt_text = example["prompt"]
    tokenized = tokenizer(
        prompt_text,
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )
    example["input_ids"] = tokenized["input_ids"]
    example["attention_mask"] = tokenized["attention_mask"]
    return example

In [61]:
test_dataset = test_data.map(
        lambda ex: tokenize_example(ex, tokenizer, max_length = 2048)
          )
test_dataset.select(range(5)).to_pandas()

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

,movie_title,matched_title,original_question,prompt,target_answer,input_ids,attention_mask
0,Mom's Got a Date With a Vampire,Mom's Got a Date With a Vampire,Who made up the spoon test?,"In the movie ""Mom's Got a Date With a Vampire""...",Adam,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,The Iron Lady,The Iron Lady,What is the Community Charge also known as?,"In the movie ""The Iron Lady""\nQuestion: What i...",Poll Tax,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,David's Birthday,David's Birthday,Where does the beach house locate?,"In the movie ""David's Birthday""\nQuestion: Whe...",At the foot of Mount Circeo.,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Nine,Nine,Luisa is promised the house will be filled wit...,"In the movie ""Nine""\nQuestion: Luisa is promis...",Flowers.,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Deadly Friend,Deadly Friend,What is the name of Paul's mother?,"In the movie ""Deadly Friend""\nQuestion: What i...",Jeannie,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
def collate_fn(batch):
    input_ids_list = [item["input_ids"] for item in batch]
    attention_mask_list = [item["attention_mask"] for item in batch]
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "prompt": [item["prompt"] for item in batch],
        "target_answer": [item["target_answer"] for item in batch],
        "movie_title": [item["movie_title"] for item in batch],
        "matched_title": [item["matched_title"] for item in batch],
        "prompt_length": [len(item["input_ids"]) for item in batch]
    }

In [64]:
batch_size = 32
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
generated_answers2 = []
start_time = time.time()
num_samples = len(test_data) 

model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            repetition_penalty=1.2,
        )
        for i in range(len(outputs)):
            prompt_len = batch["prompt_length"][i]
            full_output = outputs[i]
            generated_ids = full_output[prompt_len:] 
            generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
            if "Answer (in a concise phrase):" in generated_text:
                answer = generated_text.split("Answer (in a concise phrase):")[-1].strip()
            else:
                answer = generated_text.strip()
            generated_answers2.append(answer)
inference_time = time.time() - start_time
print(f"\nInference Time: {inference_time:.2f} seconds for {num_samples} samples")


Inference Time: 471.00 seconds for 1987 samples


In [ ]:
import evaluate
ground_truth_answers = test_dataset["target_answer"]

def compute_f1(preds, refs):
    f1_scores = []
    for pred, ref_list in zip(preds, refs):
        ref = ref_list[0] if isinstance(ref_list, list) and len(ref_list) > 0 else ""
        pred_tokens = set(pred.lower().split())
        ref_tokens = set(ref.lower().split())
        common = pred_tokens & ref_tokens
        if not pred_tokens or not ref_tokens:
            f1 = 1.0 if pred.strip().lower() == ref.strip().lower() else 0.0
        else:
            precision = len(common) / len(pred_tokens)
            recall = len(common) / len(ref_tokens)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        f1_scores.append(f1)
    return sum(f1_scores) / len(f1_scores) * 100

f1_score_value = compute_f1(generated_answers2, ground_truth_answers)
em_score = 0.0
count_em = 0

for pred, ref_list in zip(generated_answers2, ground_truth_answers):
    ref = ref_list[0] if isinstance(ref_list, list) and len(ref_list) > 0 else ""
    if pred.strip().lower() == ref.strip().lower():
        count_em += 1

em_score = count_em / len(generated_answers2) * 100
bleu = evaluate.load("bleu")
ground_truth_answers_fixed = [ref if len(ref) > 0 else [""] for ref in ground_truth_answers]
bleu_score = bleu.compute(predictions=generated_answers2, references=ground_truth_answers_fixed)
rouge = evaluate.load("rouge")
rouge_scores = rouge.compute(predictions=generated_answers2, references=ground_truth_answers_fixed)

print("\n--- Evaluation Metrics ---")
print(f"Exact Match: {em_score:.2f}%")
print(f"F1 Score: {f1_score_value:.2f}%")
print(f"BLEU Score: {bleu_score['bleu']:.4f}")
print("ROUGE Scores:")
for key, value in rouge_scores.items():
    print(f"{key}: {value:.4f}")


--- Evaluation Metrics ---
Exact Match: 0.00%
F1 Score: 0.00%
BLEU Score: 0.0000
ROUGE Scores:
rouge1: 0.0393
rouge2: 0.0011
rougeL: 0.0392
rougeLsum: 0.0395


In [ ]:
test_dataset = test_dataset.add_column("predicted_answer", generated_answers2)

In [68]:
test_dataset.select(range(1000)).to_pandas()

,movie_title,matched_title,original_question,prompt,target_answer,input_ids,attention_mask,predicted_answer
0,Mom's Got a Date With a Vampire,Mom's Got a Date With a Vampire,Who made up the spoon test?,"In the movie ""Mom's Got a Date With a Vampire""...",Adam,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The man is something like DAVIE - testing out ...
1,The Iron Lady,The Iron Lady,What is the Community Charge also known as?,"In the movie ""The Iron Lady""\nQuestion: What i...",Poll Tax,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Community charge
2,David's Birthday,David's Birthday,Where does the beach house locate?,"In the movie ""David's Birthday""\nQuestion: Whe...",At the foot of Mount Circeo.,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The Bayle home is on hand.lab - The mank coupl...
3,Nine,Nine,Luisa is promised the house will be filled wit...,"In the movie ""Nine""\nQuestion: Luisa is promis...",Flowers.,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lies people and how they'd better have no chan...
4,Deadly Friend,Deadly Friend,What is the name of Paul's mother?,"In the movie ""Deadly Friend""\nQuestion: What i...",Jeannie,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Paula few days since it was not enough to be a...
...,...,...,...,...,...,...,...,...
995,Elf,Elf,Who did Papa Elf adopt?,"In the movie ""Elf""\nQuestion: Who did Papa Elf...",an orphan,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Wally-thing. and other thing only thing that's...
996,Tuff Turf,Tuff Turf,Where did Morgan live before moving to Califor...,"In the movie ""Tuff Turf""\nQuestion: Where did ...",Connecticut,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Before what is coming (not giving place for ot...
997,Unaccompanied Minors,Unaccompanied Minors,What did Beef say they would find to make Spen...,"In the movie ""Unaccompanied Minors""\nQuestion:...",A Christmas tree,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",A.
998,Monty Python and the Holy Grail,Monty Python and the Holy Grail,WHAT DOES SIR BEDEVERE THE WISE BRING THE VILL...,"In the movie ""Monty Python and the Holy Grail""...",A WITCH,"[151654, 151654, 151654, 151654, 151654, 15165...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",the entire villi etch is being brought by men....


In [ ]:
import torch

single_prompt = """
In the movie "Deadly Friend" The plot was as follows: Teenage science genius Paul Conway (Matthew Laborteaux) and his single mother Jeannie (Anne Twomey) move into their new house in the town of Welling. He soon becomes friends with newspaper delivery boy Tom Toomey (Michael Sharrett). Living next door to Paul is the beautiful Samantha Pringle (Kristy Swanson) and her abusive, alcoholic father Harry (Richard Marcus). Paul built a robot named BB (Charles Fleischer), which occasionally displays autonomous behavior, such as being protective of Paul. Paul, Jeannie, and BB meet Paul's professor, Dr. Johanson (Russ Marin), at Polytech, a prestigious university Paul has a scholarship at. Dr. Johanson gives them a tour of the new laboratory Paul will have access to. While Tom helps Paul teach BB how to deliver newspapers, they stop at the house of reclusive harridan Elvira Parker (Anne Ramsey), who threatens the boys with a double-barreled shotgun and expresses instant dislike for BB. Walking away, the three encounter a motorcycle gang led by bully Carl (Andrew Roperto). When Carl intimidates Paul by pushing him into garbage, BB assaults him by grabbing his crotch. The gang rides away with Carl vowing revenge. Paul, Samantha, Tom, and BB begin to develop a close friendship, much to Harry's annoyance. One day, while playing basketball, BB accidentally tosses the ball onto Elvira's porch. She stomps out of her house and takes the ball, refusing to give it back, with BB taking note of Elvira's hostile attitude. On Halloween night, Samantha comes over with a bloody nose (presumably from her father's abuse) and asks for ice. Samantha goes out with Paul, Tom, and BB. Tom decides to pull a prank on Elvira. BB unlocks her gate and Samantha rings her doorbell. When alarms go off, they hide in a shrubbery nearby. When Elvira sees BB standing near her porch, she shoots him with her shotgun without hesitation, destroying him. Paul is devastated by the loss of his robotic friend. On Thanksgiving Day, Samantha joins Paul and Jeannie for dinner. Afterwards, Paul and Samantha share their first kiss. Samantha returns home late at night, outraging her father. He punches her and pushes her down the stairs. At Polytech's hospital, Paul learns that Samantha is now brain-dead and will also be on life support for 24 hours, after which the plug will be pulled. As BB's microchip can interface with the human brain, Paul decides to use it to revive Samantha. The boys enter the hospital using the key taken from Tom's father, who works as a security guard there. After Tom deactivates the power from the basement, Paul takes Samantha to his lab. He inserts the microchip into Samantha's brain and takes her back to his house, hiding her in the shed. After he activates the microchip, Samantha "wakes up", but her mannerisms are completely mechanical, suggesting BB is in control of her body. The police arrive at Samantha's home and inform Harry that her body has disappeared. In the middle of the night, Paul finds Samantha staring at the window, looking at her father, and he deactivates the microchip. The next morning, Paul awakens and finds Samantha gone. He searches for her in the street to no avail. Samantha goes back to her house and down the cellar. When Harry finds the cellar door open and goes downstairs, she attacks him, breaks his wrist and snaps his neck. Paul finds Samantha and Harry's corpse in the cellar. Horrified, he hides the body, takes Samantha back to his home and locks her in his bedroom. At night, Samantha breaks into Elvira's house and attacks her by throwing her at the wall of her living room. As a terrified Elvira screams for her life, Samantha kills her by throwing the basketball she stole from Tom at her head, causing it to explode. Elvira's headless corpse then walks around the living room until it collapses. When Tom learns of Samantha's rampage, he and Paul get into a fight. Tom threatens to call the police and end this once and for all. Still being protective of Paul, Samantha jumps out the attic window and attacks Tom, with Paul and Jeannie intervening. Trying to get her under control, Paul slaps Samantha, which results in her trying to strangle him. Samantha then lets him go and runs away. As Paul goes after her, he again encounters Carl, who gets into a fight with him. Samantha goes back for Paul, grabs Carl and throws him at an incoming police car, the impact killing him. She runs back to Paul's shed, where Paul comforts her and realizes she's regaining some of her humanity. However, the police arrive with their weapons pointed at Samantha, who yells out Paul's name in her human voice. She runs towards him, trying to protect him, with Sergeant Volchek (Lee Paul) thinking she's trying to attack him and shooting her. She says Paul's name one more time before dying in his arms. Later at the morgue, Paul tries to steal Samantha's body once more, not having learned his lesson. Suddenly, Samantha grabs Paul's neck and her face rips apart, revealing a terrifying variant of BB's head. Her skin strips away, revealing half-robotic bones underneath. With a robotic voice, Samantha tells him to come with her. When Paul refuses, she snaps his neck off-screen, killing him. Question: What is the name of Paul's mother? Answer (in a concise phrase):
"""

inputs = tokenizer(
    single_prompt,
    return_tensors="pt"   
)
input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)
model.eval()

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=20,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        repetition_penalty=1.2,
    )
    
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("=== Prompt ===\n", single_prompt)
print("\n=== Model Output ===")
print(generated_text)

=== Prompt ===
 
In the movie "Deadly Friend" The plot was as follows: Teenage science genius Paul Conway (Matthew Laborteaux) and his single mother Jeannie (Anne Twomey) move into their new house in the town of Welling. He soon becomes friends with newspaper delivery boy Tom Toomey (Michael Sharrett). Living next door to Paul is the beautiful Samantha Pringle (Kristy Swanson) and her abusive, alcoholic father Harry (Richard Marcus). Paul built a robot named BB (Charles Fleischer), which occasionally displays autonomous behavior, such as being protective of Paul. Paul, Jeannie, and BB meet Paul's professor, Dr. Johanson (Russ Marin), at Polytech, a prestigious university Paul has a scholarship at. Dr. Johanson gives them a tour of the new laboratory Paul will have access to. While Tom helps Paul teach BB how to deliver newspapers, they stop at the house of reclusive harridan Elvira Parker (Anne Ramsey), who threatens the boys with a double-barreled shotgun and expresses instant dislike

# Rag inference test

#DIY RAG test

In [ ]:
dataset_name = "ibm-research/duorc"
subset_name = "SelfRC"  #  "ParaphraseRC" 
test_dataset = load_dataset(dataset_name, subset_name, split="test")
test_subset = test_dataset.shuffle(seed=42).select(range(1000))  # 100 examples
test_subset.select(range(5)).to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.50k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60721 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12961 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12559 [00:00<?, ? examples/s]

,plot_id,plot,title,question_id,question,answers,no_answer
0,/m/06r6nf,Adam Hansen (Matt O'Leary) and his best friend...,Mom's Got a Date With a Vampire,5d9fff6a-0507-fba1-b41c-21d7ad650f9a,Who made up the spoon test?,[Adam],False
1,/m/0g9wdmc,The film begins in September 2008 (opening aga...,The Iron Lady,286ea526-555c-a2a7-807d-9b1ecd91c060,What is the Community Charge also known as?,[Poll Tax],False
2,/m/0crst6n,"Two pairs of friends, Matteo and Francesca &am...",David's Birthday,d1b01647-6d27-f3f8-beef-b566edbe86fa,Where does the beach house locate?,[At the foot of Mount Circeo.],False
3,/m/04jplwp,This article's plot summary may be too long or...,Nine,77854dc8-b9ee-adec-a1b3-d727a9c30319,Luisa is promised the house will be filled wit...,[Flowers.],False
4,/m/06z5rr,Teenage science genius Paul Conway (Matthew La...,Deadly Friend,02453a75-26d2-5742-c3e4-1df817bb67bb,What is the name of Paul's mother?,[Jeannie],False


In [ ]:
movie_to_plot = {}

for item in test_subset:
    title = item["title"]
    plot = item["plot"]
    if title in movie_to_plot:
        movie_to_plot[title] += " " + plot
    else:
        movie_to_plot[title] = plot

for title, plot in list(movie_to_plot.items())[:1]:
    print(f"Title: {title}\nPlot snippet: {plot[:300]}...\n")

Title: Mom's Got a Date With a Vampire
Plot snippet: Adam Hansen (Matt O'Leary) and his best friend Duffy (Jake Epstein) have some tickets for the Headless Horseman concert, and his sister Chelsea (Laura Vandervoort) has a date with her dreamy boyfriend Peter. The only problem is, they are both grounded: Adam is punished because he did not do his home...



In [ ]:
!pip install nltk tqdm -q

In [ ]:
import nltk
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

def split_into_sentences(text):
    return sent_tokenize(text)

example_title = list(movie_to_plot.keys())[0]
sentences = split_into_sentences(movie_to_plot[example_title])
print("Example sentences from the plot:")
for s in sentences[:3]:
    print("-", s)

Example sentences from the plot:
- Adam Hansen (Matt O'Leary) and his best friend Duffy (Jake Epstein) have some tickets for the Headless Horseman concert, and his sister Chelsea (Laura Vandervoort) has a date with her dreamy boyfriend Peter.
- The only problem is, they are both grounded: Adam is punished because he did not do his homework, instead making up a story using an article from the magazine The Weekly Secret, and Chelsea because she called Adam a dweeb, which their mother, Lynette (Caroline Rhea), happened to hear.
- Chelsea and Adam will do whatever it takes to get their mother out of the house, even if it includes a chance meeting with a very mysterious man named Dimitri.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def retrieve_relevant_sentence(movie_title, question):
    if movie_title not in movie_to_plot:
        return f"No plot found for movie title '{movie_title}'."

    plot = movie_to_plot[movie_title]
    sentences = split_into_sentences(plot)

    vectorizer = TfidfVectorizer().fit(sentences + [question])
    sentence_vectors = vectorizer.transform(sentences)
    question_vector = vectorizer.transform([question])

    similarities = cosine_similarity(sentence_vectors, question_vector).flatten()

    best_idx = int(np.argmax(similarities))
    best_sentence = sentences[best_idx]
    return best_sentence

example_movie = example_title  
example_question = "Who made up the spoon test?"
retrieved_sentence = retrieve_relevant_sentence(example_movie, example_question)
print("Retrieved sentence:")
print(retrieved_sentence)


Retrieved sentence:
Adam and Taylor make the vampire do the spoon test (a fake test made up by Adam to get Taylor to stop calling Dimitri a vampire).


In [ ]:
def answer_query(input_text):
    import re
    title_match = re.search(r'In the movie\s+"([^"]+)"', input_text)
    question_match = re.search(r'Question:\s*(.*)', input_text)

    if not title_match or not question_match:
        return "Input format not recognized. Please use the specified format."

    movie_title = title_match.group(1)
    question_text = question_match.group(1).split("Answer")[0].strip()
    context_sentence = retrieve_relevant_sentence(movie_title, question_text)
    return context_sentence

input_text = '''In the movie "Mom's Got a Date With a Vampire"
Question: Who made up the spoon test?
Answer (in a concise phrase):'''
print("Final Answer:", answer_query(input_text))


Final Answer: Adam and Taylor make the vampire do the spoon test (a fake test made up by Adam to get Taylor to stop calling Dimitri a vampire).


# Vector RAG test


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
movie_titles = list(movie_to_plot.keys())
title_embeddings = model.encode(movie_titles, convert_to_tensor=True)

def find_best_title(query_title):
    query_embedding = model.encode(query_title, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, title_embeddings)[0]
    best_idx = int(np.argmax(cosine_scores))
    return movie_titles[best_idx]

input_title = "Mom's got a date with a Vampire"  
matched_title = find_best_title(input_title)
print("Matched Title:", matched_title)

def get_best_sentence(movie_title, question):
    plot_text = movie_to_plot[movie_title]
    sentences = split_into_sentences(plot_text)
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    question_embedding = model.encode(question, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(question_embedding, sentence_embeddings)[0]
    best_idx = int(np.argmax(cosine_scores))
    return sentences[best_idx]

input_question = "Who made up the spoon test?"
best_sentence = get_best_sentence(matched_title, input_question)
print("Retrieved Sentence:", best_sentence)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Matched Title: Mom's Got a Date With a Vampire
Retrieved Sentence: Adam and Taylor make the vampire do the spoon test (a fake test made up by Adam to get Taylor to stop calling Dimitri a vampire).
